In [28]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mlsp-padded/reshaped/x_train548.npy
/kaggle/input/mlsp-padded/reshaped/x_train224.npy
/kaggle/input/mlsp-padded/reshaped/x_train768.npy
/kaggle/input/mlsp-padded/reshaped/x_train607.npy
/kaggle/input/mlsp-padded/reshaped/x_train1.npy
/kaggle/input/mlsp-padded/reshaped/x_train60.npy
/kaggle/input/mlsp-padded/reshaped/x_train945.npy
/kaggle/input/mlsp-padded/reshaped/x_train158.npy
/kaggle/input/mlsp-padded/reshaped/x_train706.npy
/kaggle/input/mlsp-padded/reshaped/x_train428.npy
/kaggle/input/mlsp-padded/reshaped/x_train229.npy
/kaggle/input/mlsp-padded/reshaped/x_train305.npy
/kaggle/input/mlsp-padded/reshaped/x_train578.npy
/kaggle/input/mlsp-padded/reshaped/x_train990.npy
/kaggle/input/mlsp-padded/reshaped/x_train950.npy
/kaggle/input/mlsp-padded/reshaped/x_train366.npy
/kaggle/input/mlsp-padded/reshaped/x_train605.npy
/kaggle/input/mlsp-padded/reshaped/x_train854.npy
/kaggle/input/mlsp-padded/reshaped/x_train620.npy
/kaggle/input/mlsp-padded/reshaped/x_train290.npy
/ka

In [56]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from numpy import array
from numpy import argmax
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix
#from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import os
import librosa
    
def get_data():
    lst_names = os.listdir("../input/test-set/Audio_Classification-MLSP-test")
    x_train = []
    for m in lst_names:
        x_train.append(np.load("../input/test-set/Audio_Classification-MLSP-test/" + m))
    with open("../input/mlsp-padded/reshaped/Labels_dict") as f:
        dict1 = json.loads(f.read())
    return x_train, lst_names, dict1#, y_train

def reshape_data(lst_x):#, lst_y):
    for b in range(len(lst_x)):
        lst_x[b] = lst_x[b].tolist()
    count = 0
    for m in range(len(lst_x)):
        ct = len(lst_x[m][0][0])
        if ct > count:
            count = ct
    for n in range(len(lst_x)):
        num_zeros = count - len(lst_x[n][0][0])
        zero = [0]*num_zeros
        for m in range(len(lst_x[n][0])):
            lst_x[n][0][m].extend(zero)
    del(zero)
    x_train=[]
    for m in range(len(lst_x)):
        x_train.append(lst_x[m][0])
    x_train = np.array(x_train)
    del(lst_x)
    x_train = x_train[..., np.newaxis]
    #x_test = x_test[..., np.newaxis]
    return x_train#, y_train


In [57]:
x_test, lst_names, dict1 = get_data()

In [58]:
x_test = reshape_data(x_test)#, y_test = reshape_data(x_test, y_test)

In [59]:
x_test.shape

(201, 128, 2584, 1)

In [44]:
model1 = tf.keras.models.load_model('../input/models/model_1.h5')

In [60]:
model1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 1292, 32)      320       
_________________________________________________________________
batch_normalization (BatchNo (None, 64, 1292, 32)      128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 646, 32)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 646, 32)       128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 646, 64)       18496     
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 646, 64)       256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 323, 64)       0

In [61]:
yhat_probs = model1.predict(x_test, verbose=1)
y_hat=np.argmax(yhat_probs,axis=1)
print(y_hat)

7/7 [==============================] - 0s 29ms/step
[6 9 0 3 8 5 7 2 2 5 2 8 4 7 3 6 7 8 8 4 7 7 8 2 3 7 4 1 0 1 6 2 8 2 3 8 0
 3 0 6 8 4 5 5 3 9 4 5 0 2 6 6 2 1 8 3 7 1 4 8 6 2 2 3 0 9 2 8 8 4 4 7 2 0
 2 7 3 8 0 8 9 7 9 5 9 2 5 2 6 9 1 5 7 2 0 7 5 3 8 9 2 1 1 2 2 4 3 5 4 4 7
 2 2 0 5 7 1 3 9 4 1 7 1 6 6 6 0 6 4 1 2 3 4 4 1 0 1 0 3 9 9 8 0 7 7 3 2 0
 6 4 8 6 1 7 8 6 1 6 8 2 5 6 6 7 3 6 7 7 7 8 1 2 6 1 4 6 6 4 6 3 8 2 0 1 4
 3 1 5 6 2 6 2 5 8 9 7 3 1 3 4 0]


In [62]:
print(len(y_hat))

201


In [63]:
lst = []
for m in y_hat:
    for key, value in dict1.items():
        if value == m:
            #print(key)
            lst.append(key)    

In [64]:
print(len(lst))

201


In [65]:
df = pd.DataFrame(list(zip(lst_names, lst)),
               columns =['fileid', 'prediction'])

In [66]:
df.head()

,fileid,prediction
0,MLSP_test1443477.npy,Microwave_oven
1,MLSP_test1602453.npy,Walk_and_footsteps
2,MLSP_test463146.npy,Bark
3,MLSP_test1810394.npy,Shatter
4,MLSP_test616123.npy,Doorbell


In [67]:
df.to_csv("model_1_test_result.csv")

In [68]:
del(model1)

**MODEL 5**

In [ ]:
model5 = tf.keras.models.load_model('../input/models/model_5.h5')

In [70]:
model5.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 1292, 32)      320       
_________________________________________________________________
batch_normalization (BatchNo (None, 64, 1292, 32)      128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 646, 32)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 646, 32)       128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 646, 64)       18496     
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 646, 64)       256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 323, 64)       0

In [71]:
yhat_probs = model5.predict(x_test, verbose=1)
y_hat=np.argmax(yhat_probs,axis=1)
print(y_hat)
print(len(y_hat))

lst = []
for m in y_hat:
    for key, value in dict1.items():
        if value == m:
            #print(key)
            lst.append(key) 

7/7 [==============================] - 0s 29ms/step
[6 9 0 3 8 5 7 4 9 9 2 8 4 7 3 6 5 8 3 4 7 7 8 7 3 7 4 1 0 1 0 0 8 2 3 8 0
 3 0 0 8 4 5 5 3 9 4 5 0 2 6 6 2 5 8 3 7 1 4 8 1 2 2 3 0 9 2 8 8 4 8 7 5 1
 2 7 3 3 0 2 9 7 9 5 4 2 5 2 6 9 1 5 2 9 0 0 5 3 8 4 2 1 1 2 2 4 3 5 4 4 2
 0 0 0 5 0 7 6 9 4 1 8 1 6 4 0 0 1 4 1 2 3 4 4 1 0 9 4 3 9 9 0 0 7 7 3 2 0
 5 4 8 6 7 0 0 6 1 6 4 2 5 4 6 7 3 7 4 7 7 8 1 2 6 1 4 6 6 4 4 3 8 2 0 1 9
 3 1 5 6 2 6 2 5 8 9 7 3 5 3 4 0]
201


In [72]:
df5 = pd.DataFrame(list(zip(lst_names, lst)),columns =['fileid', 'prediction'])
df5.head()
df.to_csv("model_5_test_result.csv")

In [73]:
del(model5)

**MODEL 2**

In [ ]:
model2 = tf.keras.models.load_model('../input/models/model_2.h5')

In [ ]:
model2.summary()

In [ ]:
yhat_probs = model2.predict(x_test, verbose=1)
y_hat=np.argmax(yhat_probs,axis=1)
print(y_hat)
print(len(y_hat))

lst = []
for m in y_hat:
    for key, value in dict1.items():
        if value == m:
            #print(key)
            lst.append(key) 

In [ ]:
df2 = pd.DataFrame(list(zip(lst_names, lst)),columns =['fileid', 'prediction'])
df2.head()
df.to_csv("model_2_test_result.csv")

In [ ]:
del(model2)
del(x_test)

**MODEL 3**

In [91]:
model3 = tf.keras.models.load_model('../input/models/model_3.h5')

In [92]:
def get_data_features(n_mfcc):
    x_test = []
    lst_names = os.listdir("../input/test-set/Audio_Classification-MLSP-test")
    for m in lst_names:
        z = np.load("../input/test-set/Audio_Classification-MLSP-test/" + m)
        features = librosa.feature.mfcc(S=librosa.power_to_db(z), n_mfcc = 16)
        x_test.append(features)
    return x_test, lst_names

In [93]:
import librosa
n_mfcc = 16
x_test, lst_names = get_data_features(n_mfcc)

In [94]:
x_test = reshape_data(x_test)

In [95]:
(temp, f, t, num) = x_test.shape
print(f)
print(t)
print(num)
print(x_test.shape)

16
2584
1
(201, 16, 2584, 1)


In [96]:
model3.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 8, 1292, 32)       320       
_________________________________________________________________
batch_normalization_7 (Batch (None, 8, 1292, 32)       128       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 4, 646, 32)        0         
_________________________________________________________________
batch_normalization_8 (Batch (None, 4, 646, 32)        128       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 4, 646, 64)        18496     
_________________________________________________________________
batch_normalization_9 (Batch (None, 4, 646, 64)        256       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 2, 323, 64)       

In [97]:
yhat_probs = model3.predict(x_test, verbose=1)
y_hat=np.argmax(yhat_probs,axis=1)
print(y_hat)
print(len(y_hat))

lst = []
for m in y_hat:
    for key, value in dict1.items():
        if value == m:
            #print(key)
            lst.append(key)

7/7 [==============================] - 0s 4ms/step
[1 0 0 0 8 5 7 0 0 0 0 8 0 7 0 0 7 8 0 0 7 7 0 0 0 0 4 0 0 0 1 2 8 0 0 8 0
 0 0 0 8 0 0 0 3 0 0 0 0 2 0 0 2 0 6 6 7 0 0 8 1 2 2 3 2 0 7 8 0 0 6 0 2 0
 7 0 0 8 0 0 9 0 0 0 0 2 0 2 0 2 0 7 0 0 0 0 7 0 8 0 0 1 0 2 0 7 3 3 0 6 0
 0 0 0 0 0 7 0 0 0 0 0 7 0 0 0 0 0 0 0 0 0 0 4 0 0 0 0 3 0 7 0 0 7 0 6 2 0
 1 2 0 0 0 0 0 0 0 2 0 0 0 6 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 6 3 8 0 0 0 0
 0 0 0 0 7 0 0 0 8 0 0 0 0 0 0 0]
201


In [84]:
df5 = pd.DataFrame(list(zip(lst_names, lst)),columns =['fileid', 'prediction'])
df5.head()
df.to_csv("model_3_test_result.csv")

In [85]:
del(model3)

**MODEL 4**

In [87]:
model4.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_9 (Flatten)          (None, 41344)             0         
_________________________________________________________________
dense_35 (Dense)             (None, 1024)              42337280  
_________________________________________________________________
dense_36 (Dense)             (None, 256)               262400    
_________________________________________________________________
dense_37 (Dense)             (None, 64)                16448     
_________________________________________________________________
batch_normalization_18 (Batc (None, 64)                256       
_________________________________________________________________
dropout_9 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_38 (Dense)             (None, 10)               

In [88]:
yhat_probs = model4.predict(x_test, verbose=1)
y_hat=np.argmax(yhat_probs,axis=1)
print(y_hat)
print(len(y_hat))

lst = []
for m in y_hat:
    for key, value in dict1.items():
        if value == m:
            #print(key)
            lst.append(key)

7/7 [==============================] - 0s 3ms/step
[0 6 6 3 8 3 7 6 6 3 3 3 0 3 3 6 6 3 3 6 6 7 3 3 3 0 6 3 0 3 6 6 8 3 3 3 0
 3 3 0 3 3 3 3 3 6 3 3 0 3 3 3 0 3 6 6 3 3 3 8 6 3 3 6 3 3 6 3 0 6 3 6 3 3
 3 3 3 3 0 0 3 6 3 3 6 3 8 3 6 3 3 6 6 3 0 3 6 3 8 6 0 3 3 6 3 3 3 3 3 6 3
 0 3 0 3 3 3 3 6 3 3 6 3 3 6 3 0 3 6 3 3 3 3 6 3 3 3 3 3 3 3 8 0 0 3 3 3 0
 6 3 0 6 3 3 0 3 3 3 3 3 3 3 6 3 6 3 6 3 3 3 3 3 6 3 6 6 3 3 3 3 3 3 8 3 6
 3 3 3 6 3 3 0 3 8 6 6 3 3 3 3 0]
201


In [89]:
df4 = pd.DataFrame(list(zip(lst_names, lst)),columns =['fileid', 'prediction'])
df4.head()
df.to_csv("model_4_test_result.csv")

In [ ]:
del(model4)